# Examen Bimestral 
**Nombre:** Rossy Armendariz

# Obtención de database
Se carga los dos archivos de la database:

In [15]:
import pandas as pd

critics_path = "../input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv"
movies_path = "../input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_movies.csv"

critics_df = pd.read_csv(critics_path)
movies_df = pd.read_csv(movies_path)

# Preprocesamiento de DatosSe realiza el preprocesamiento de datos para lo cual se cargan las librerías necesarias como:
- nltk para stemming. 
- re para eliminar caracteres especiales

In [16]:
import re
from nltk.stem import PorterStemmer

Se inicializa el componente para stemming.

In [17]:
stemmer = PorterStemmer()

Se carga el archivo stopwords para la eliminación de estas.

In [18]:
# Ruta al archivo de stopwords en la estructura de Kaggle
stopwords_path = "../input/stopwords/stopwords"

# Cargar las stopwords desde el archivo
with open(stopwords_path, 'r') as file:
    stop_words = set(file.read().splitlines())

Se realiza la función preprocess_text para el preprocesamiento del texto donde se hace lo siguiente:
* Pasar a minúsculas
* Eliminación de caracteres especiales
* Tokenización
* Eliminación de stopwords
* Stemming

In [23]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Si no es una cadena de texto, devolvemos una cadena vacía
    text = text.lower()  # Pasar a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Eliminación caracteres especiales
    tokens = text.split()  # Tokenización
    processed_tokens = [word for word in tokens if word not in stop_words]  # Eliminación de stopwords
    processed_tokens = [stemmer.stem(word) for word in processed_tokens] # Stemming
    return ' '.join(processed_tokens)

critics_df['processed_review'] = critics_df['review_content'].apply(preprocess_text)

# Construcción del Sistema
Para la construcción del sistema se hace la vectorización mediante TF-IDF donde se usa la libreria "from sklearn.feature_extraction.text import TfidfVectorizer" de la database critics.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(critics_df['processed_review'])

La función search_reviews busca las críticas más relevante para la consulta realizada. Además, se hace el preprocesamiento de la query, se vectoriza de igual forma que se realizó con el texto. Por otro lado, se calcula las similitudes cosenos para buscar los resultados más relevantes.

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

def search_reviews(query, top_n=5):

    # Preprocesamiento y vectorización de la consulta
    preprocessed_query = preprocess_text(query)  # Preprocesar la consulta
    query_vector = vectorizer.transform([preprocessed_query])  # Vectorización de la consulta
    
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()  # Calcular similitudes cosenos
    
    # Ranking
    top_indices = similarities.argsort()[-top_n:][::-1]
    
    # dataframe con la información de los 5 críticas más relevantes en base al calculo de similitud coseno
    results = critics_df.iloc[top_indices][['rotten_tomatoes_link', 'critic_name', 'review_content']].copy()
    results['similarity'] = similarities[top_indices]  # Se agrega el valor de similitud coseno
    
    return results

La función get_detailed_results une la información de las críticas con las de las películas en base al rotten_tomatoes_link que es el id.

In [60]:
def get_detailed_results(results):

    # Unir los resultados en base rotten_tomatoes_link
    detailed_results = results.merge(
        movies_df[['rotten_tomatoes_link', 'movie_title']],
        on='rotten_tomatoes_link',
        how='left'
    )
    return detailed_results[['rotten_tomatoes_link', 'movie_title', 'critic_name', 'review_content', 'similarity']]

# Simulación de Consultas

Para la similución de consultas se inge el query y se realiza la búsqueda en las críticas y se agrega la información de la películas encontradas.

In [61]:
queries = ["space travel", "watch with family"]

for query in queries:
    print(f"\nConsulta: {query}")
    
    review_results = search_reviews(query) # Buscar críticas relevantes
    detailed_results = get_detailed_results(review_results) # Agregar detalles de las películas

    print(detailed_results)


Consulta: space travel
       rotten_tomatoes_link        movie_title       critic_name  \
0  m/the_trip_to_spain_2017  The Trip to Spain    Robin Clifford   
1               m/first_man          First Man      Robert Roten   
2                m/ad_astra           Ad Astra  Michael J. Casey   
3  m/a_wrinkle_in_time_2018  A Wrinkle in Time    Marisa Carpico   
4        m/captain_abu_raed   Captain Abu Raed          Tom Long   

                                      review_content  similarity  
0  You like traveling with these guys, but someti...    0.770067  
1  The special effects depicting space travel in ...    0.724362  
2  Ad Astra isn't so much about the science of sp...    0.711242  
3  A Wrinkle in Time is about traveling through t...    0.681537  
4  His journey makes for a complex film that goes...    0.587867  

Consulta: watch with family
          rotten_tomatoes_link                 movie_title    critic_name  \
0                   m/rock_dog                    Rock Dog 

# Análisis de Resultados
En base a los resultados obtenidos, se puede analizar que el método seleccionado fue eficiente para las búsquedas realizadas. En el caso de la consulta space travel, se puede visualizar que los resultados obtenidos si estan relacionados con viajes o el espacio según lo que se encuentra en el contenido de la crítica y se ordena en base al orden la similitud. De igual forma, papra la consulta watch in family, se puede visualizar en el contenido del review content que nos habla que se puede observar en familia o que la película se trata sobre una familia. Aunque esto puede afectar en el caso que no se una pelicula para ver en familia pero que se trate sobre un familia.